In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.models import load_model
from keras import applications
from keras import optimizers
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
import pickle
from keras import regularizers

import keras.backend as K

from model import Model
K.clear_session()

In [6]:
num_classes = 10
ns = 10 # number of shadow models
ds = 50 # number of data points per shadow model for training
batch_size = 128
epochs = 5
lrate = 0.001
decay = 1e-7

In [4]:
X_attack = np.load('data/X_attack.npy')
y_attack = np.load('data/y_attack.npy')
y_attack_not_one_hot = y_attack.copy()
y_attack = keras.utils.to_categorical(y_attack, num_classes)

print(X_attack.shape, y_attack.shape)

(30000, 28, 28, 1) (30000, 10)


In [ ]:
X_train_attack = np.zeros((2*ds*ns,num_classes))
y_train_attack = np.zeros((2*ds*ns,))
labels_attack_train = np.zeros((2*ds*ns,num_classes))

for j in np.arange(ns):
    
    indices = np.arange(X_attack.shape[0])
    np.random.seed(j)
    np.random.shuffle(indices)
    X_member_shadow = X_attack[indices[:ds]]
    y_member_shadow = y_attack[indices[:ds]]
    X_nonmember_shadow = X_attack[indices[ds:2*ds]]
    y_nonmember_shadow = y_attack[indices[ds:2*ds]]
    
    shadow = Model(input_shape=X_attack.shape[1:], num_classes=num_classes)
    shadow.train(X_member_shadow, y_member_shadow, batch_size, epochs, lrate, decay, X_nonmember_shadow, y_nonmember_shadow)
    print('\nFor shadow model %d'%j)
    shadow.eval()
    model_name = 'mnist_shadow_'+str(ds)+'_'+str(j)+'.h5'
    shadow.save("models/shadow_models/"+model_name)
    
    
    y_pred_member = shadow.predict(X_member_shadow)
    y_pred_nonmember = shadow.predict(X_nonmember_shadow)
    X_train_attack[j*2*ds:(j+1)*2*ds] = np.vstack((y_pred_member,y_pred_nonmember))
    y_train_attack[j*2*ds:(2*j+1)*ds] = 1
    labels_attack_train[j*2*ds:(j+1)*2*ds] = np.vstack((y_member_shadow,y_nonmember_shadow))

X_attack_train_dict = {'X_train_attack':X_train_attack,'y_train_attack':y_train_attack,'labels_attack_train':labels_attack_train}
fname = 'data/attack_train_data_'+str(ds)
np.save(fname,X_attack_train_dict)

In [8]:
C_test = [0.1,1,10]
gam_test = [0.001,0.01,0.1]
bestCgam = []

In [ ]:
use_condensed = False

In [ ]:
def getfilename(datatype,ds):
    if datatype == 'train':
        name = 'data/attack_'+datatype+'_data_'+str(ds)+'.npy'
    elif datatype == 'test': 
        if use_condensed:
            name = 'data/attack_'+datatype+'_data_condensed.npy'
        else:
            name = 'data/attack_'+datatype+'_data.npy'
    elif datatype == 'result':
        name = 'results/attack_'+datatype+'_'+str(ds)+'.npy'
    return name

def getmodelname(classi,ds):
    name = 'models/attack_models/attack_model_'+str(ds)+'_class_'+str(classi)+'.p'
    return name

In [ ]:
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

train_dict = np.load(getfilename('train',ds), allow_pickle=True).item()
test_dict = np.load(getfilename('test',ds), allow_pickle=True).item()

labels_attack_train = train_dict['labels_attack_train']
X_train_attack = train_dict['X_train_attack']
y_train_attack = train_dict['y_train_attack']

labels_attack_test = test_dict['labels_attack_test']
X_test_attack = test_dict['X_test_attack']
y_test_attack = test_dict['y_test_attack']

for i in range(num_classes):
    ind_train = np.where(labels_attack_train[:,i]==1)[0]
    X_train_list.append(X_train_attack[ind_train])
    y_train_list.append(y_train_attack[ind_train])
    
    ind_test = np.where(labels_attack_test[:,i]==1)[0]
    X_test_list.append(X_test_attack[ind_test])
    y_test_list.append(y_test_attack[ind_test])

class_c_g = np.zeros((10,2))
prec = []
recall = []
acc_classwise = []
yhat_full = np.zeros(y_test_attack.shape[0])
y_full = np.zeros(y_test_attack.shape[0])
start = 0
for i in range(num_classes):
    x = X_train_list[i]
    y = y_train_list[i]
    
    xtest = X_test_list[i]
    ytest = y_test_list[i]
    ntest = xtest.shape[0]
    nfold = 5
    kf = KFold(n_splits=nfold, shuffle=True)
    acc = np.zeros((3,3,nfold))
    for ifold, ind in enumerate(kf.split(x)):
        # Get the training data in the split
        Itr,Its = ind
        xtr = x[Itr,:]
        ytr = y[Itr]
        xts = x[Its,:]
        yts = y[Its]
        for ic,c in enumerate(C_test):
            for ig,g in enumerate(gam_test):
                svc = svm.SVC(probability=False,  kernel="rbf", C=c, gamma=g,verbose=10)
                svc.fit(xtr,ytr)
                yhat_ts = svc.predict(xts)
                acc[ic,ig,ifold] = np.mean(yhat_ts == yts)
    
    acc1 = np.mean(acc,axis=2)
    ci = np.argmax(np.amax(acc1,axis=1))
    gi = np.argmax(np.amax(acc1,axis=0))
    class_c_g[i,0] = C_test[ci]
    class_c_g[i,1] = gam_test[gi]
    
    #Now creating the actual attack classifier for class i and data_size ds
    svc = svm.SVC(probability=False,  kernel="rbf", C=C_test[ci], gamma=gam_test[gi], verbose=10)
    svc.fit(x,y)
    yhat_test = svc.predict(xtest)
    acci = np.mean(yhat_test == ytest)
    preci,reci,_,_= precision_recall_fscore_support(ytest,yhat_test,average='binary')
    prec.append(preci)
    recall.append(reci)
    acc_classwise.append(acci)
    modelname = getmodelname(i,ds)
    with open( modelname, "wb" ) as fp:
        pickle.dump( [svc, C_test[ci], gam_test[gi]], fp)
    
    y_full[start:(start+ntest)] = ytest
    yhat_full[start:(start+ntest)] = yhat_test
    
    start = start+ntest

acctotal = np.mean(yhat_full == y_full)
prectotal,recalltotal,_,_= precision_recall_fscore_support(y_full,yhat_full,average='binary')
fname = getfilename('results',ds)
dict_to_save = {'prectotal':prectotal,
                'acctotal':acctotal,
                'recalltotal':recalltotal,
                'prec':prec,
                'acc':acc_classwise,
                'recall':recall,
                'class_c_g':class_c_g}
np.save(fname,dict_to_save)
print('For ds = ',ds)
print('Attack Precision: ',prectotal)
print('Attack Recall: ',recalltotal)
print('Attack Accuracy: ',acctotal)